# <img src="https://github.com/JuliaLang/julia-logo-graphics/raw/master/images/julia-logo-color.png" height="100" /> _HackBU Workshop_

## RUN THE FOLLOWING TO DOWNLOAD JULIA:


In [ ]:
%%shell
set -e

#---------------------------------------------------#
JULIA_VERSION="1.8.2" # any version ≥ 0.7.0
JULIA_PACKAGES="IJulia BenchmarkTools"
JULIA_PACKAGES_IF_GPU="CUDA" # or CuArrays for older Julia versions
JULIA_NUM_THREADS=2
#---------------------------------------------------#

if [ -z `which julia` ]; then
  # Install Julia
  JULIA_VER=`cut -d '.' -f -2 <<< "$JULIA_VERSION"`
  echo "Installing Julia $JULIA_VERSION on the current Colab Runtime..."
  BASE_URL="https://julialang-s3.julialang.org/bin/linux/x64"
  URL="$BASE_URL/$JULIA_VER/julia-$JULIA_VERSION-linux-x86_64.tar.gz"
  wget -nv $URL -O /tmp/julia.tar.gz # -nv means "not verbose"
  tar -x -f /tmp/julia.tar.gz -C /usr/local --strip-components 1
  rm /tmp/julia.tar.gz

  # Install Packages
  nvidia-smi -L &> /dev/null && export GPU=1 || export GPU=0
  if [ $GPU -eq 1 ]; then
    JULIA_PACKAGES="$JULIA_PACKAGES $JULIA_PACKAGES_IF_GPU"
  fi
  for PKG in `echo $JULIA_PACKAGES`; do
    echo "Installing Julia package $PKG..."
    julia -e 'using Pkg; pkg"add '$PKG'; precompile;"' &> /dev/null
  done

  # Install kernel and rename it to "julia"
  echo "Installing IJulia kernel..."
  julia -e 'using IJulia; IJulia.installkernel("julia", env=Dict(
      "JULIA_NUM_THREADS"=>"'"$JULIA_NUM_THREADS"'"))'
  KERNEL_DIR=`julia -e "using IJulia; print(IJulia.kerneldir())"`
  KERNEL_NAME=`ls -d "$KERNEL_DIR"/julia*`
  mv -f $KERNEL_NAME "$KERNEL_DIR"/julia  

  echo ''
  echo "Successfully installed `julia -v`!"
  echo "Please reload this page (press Ctrl+R, ⌘+R, or the F5 key) then"
  echo "jump to the 'Checking the Installation' section."
fi

# Checking the Installation
The `versioninfo()` function should print your Julia version and some other info about the system:

In [2]:
versioninfo()

Julia Version 1.8.2
Commit 36034abf260 (2022-09-29 15:21 UTC)
Platform Info:
  OS: Linux (x86_64-linux-gnu)
  CPU: 2 × Intel(R) Xeon(R) CPU @ 2.20GHz
  WORD_SIZE: 64
  LIBM: libopenlibm
  LLVM: libLLVM-13.0.1 (ORCJIT, broadwell)
  Threads: 2 on 2 virtual cores
Environment:
  LD_LIBRARY_PATH = /usr/local/nvidia/lib:/usr/local/nvidia/lib64
  LD_PRELOAD = /usr/lib/x86_64-linux-gnu/libtcmalloc.so.4
  JULIA_NUM_THREADS = 2


In [3]:
using BenchmarkTools

M = rand(2^11, 2^11)

@btime $M * $M;

  480.047 ms (2 allocations: 32.00 MiB)


In [4]:
try
    using CUDA
catch
    println("No GPU found.")
else
    run(`nvidia-smi`)
    # Create a new random matrix directly on the GPU:
    M_on_gpu = CUDA.CURAND.rand(2^11, 2^11)
    @btime $M_on_gpu * $M_on_gpu; nothing
end

No GPU found.


<img src="https://raw.githubusercontent.com/JuliaLang/julia-logo-graphics/master/images/julia-logo-mask.png" height="100" />

---
# WORKSHOP STARTS HERE

---

## The Very Basics

First thing with any new language: the "Hello World" program. In Julia, we use `print` to print text to the console.

In [5]:
print("Hello World")

Hello World

Unlike similar languages like python, starting a new print statement does not automatically go to a new line. 

In [6]:
print("Hello World")
print("New line ?")

Hello WorldNew line ?

Julia offers us the same option as Python, where we can add `\n` to the end of a line in order to start a new line.

In [7]:
print("Hello World\n")
print("New line ?")

Hello World
New line ?

Julia also has another print option, `println`, to print with a new line at the end automatically.

In [8]:
println("Hello World!")
print("New line ?")

Hello World!
New line ?

Something included in Python is the `type()` function, which allows you to output a variable's type. Julia has this too!

In [9]:
val = 42
println(typeof(val))

pi_val = 3.1415926535897932384626433832
println(typeof(pi_val))

# pi is a built-in variable in Julia!
println(pi)
println(typeof(pi))

Int64
Float64
π
Irrational{:π}


### Comments

In Julia, short comments are done with a `#` symbol (the same way as python.) Long comments are done with `#=` to start and `=#` to end.

In [10]:
# Single comment
# println("Test 0")
println("Test 1")
#=


println("Test 2")


=#
println("Test 3")

Test 1
Test 3


### Unique Variables

One of the advantages to Julia is that you have very unique options for what you can call variables.

You can call variables anything that has a unicode value. One fun example of this is with emojis.

In [11]:
😝 = "emoji!"
println(😝)

print(typeof(😝))

emoji!
String

In [12]:
😢 = -5
😐 = 0
😄 = 5

println(😢 + 😄 == 😐)

true


While this is fun, emojis don't have too many practical uses. 

Something that is practical: many researchers need to include mathematical formulas in their papers. Anyone who has had to convert math to code or code to math (or both) will tell you that it can be somewhat difficult to keep track of your variables. Julia makes this significantly easier by allowing you to use mathematical symbols and greek letters as variables!

Some common variables you might have seen in your math or physics classes:

In [13]:
Δ = "delta"
println(Δ)

Σ = "sigma"
println(Σ)

Φ = "phi"
println(Φ)

Ψ = "psi"
println(Ψ)

α = "alpha"
println(α)

∫ = "integral"
println(∫)

∬ = "double integral"
println(∬)

∯ = "surface integral"
println(∯)

delta
sigma
phi
psi
alpha
integral
double integral
surface integral


### The triple equal signs

Something Julia borrows from JavaScript is the triple equal sign operator. A double equal sign (`==`) tests for equality, meaning simply "are these two values equal to each other, regardless of type?"

By this logic, the following should both print "true", because 1.0 is equivalent to 1 mathematically.

In [14]:
println(1.0 == 1)
println(1.0 == 1.0)

true
true


However, the triple equal sign, `===`, asks the computer to include a variable's type when checking for equality. To a computer, the values `1` and `1.0` are not equal to each other. One is an integer, the other is a float. They have different memory requirements. To a computer, these two values are not necessarily equivalent.

By this logic, only one of these should be true:

In [15]:
println(1.0 === 1)
println(1.0 === 1.0)

false
true


### Strings

String manipulation in Julia will likely look familiar if you know python.

First, strings are saved as indexable arrays, similarly to python. (Spaces are valid locations). Unlike Python (and most other programming languages,) arrays start at 1 as opposed to 0. Julia was designed to be intuitive to anyone, including people who are not very familiar with programming. Arrays also start at 1 in R and MATLAB for this reason.

In [16]:
my_str = "test string"

println(my_str[1])
println(my_str[5]) # space character
println(my_str[6])

t
 
s


Julia also includes a `length()` function similarly to python. 

In [17]:
println(length(my_str))

11


Not every string character in Julia is valid. Because Julia allows you to use any unicode value, some issues arise when trying to index into strings.

If you include a unicode character `\u2200` in a string, the entire unicode value is indexable by `str[1]`. However, if you try to then index into `str[2]`, Julia doesn't recognize this since it's in the middle of what it considers to be a single character.

The following code should print out a unicode value on the first line, and then give an error.

In [18]:
my_str_2 = "\u2200 test"

println(my_str_2[1])
println(my_str_2[2])

∀


LoadError: ignored

Part of that error includes informing you of what the next valid index is: index 4. This should print a space:

In [19]:
println(my_str_2[4])

Since this can be very unintuitive, Julia has very convenient functions built-in to help with finding valid indices.

Some of these include finding the first and last indices, as well as next and previous valid indices. It also has a function to find all valid indices in a function.

`firstindex()`
`lastindex()`
`nextind()`
`prevind()`
`eachindex()`

Some of these are demonstrated below:

In [20]:
println(firstindex(my_str_2))
println(lastindex(my_str_2))

println("------------------")

collect(eachindex(my_str_2))

1
8
------------------


6-element Vector{Int64}:
 1
 4
 5
 6
 7
 8

Notice how `collect(eachindex))` excluded characters 2 and 3 since those are part of the unicode value.

Fortunately, Julia is also intelligent enough to know that if you are using the string as an iterable object, it should just skip these automatically.

In [21]:
for char in my_str_2
  println(char)
end

∀
 
t
e
s
t


Julia also offers easy ways of concatinating strings. While Julia offers multiple ways of doing this, the main one is `*`. Many similar languages use `+`, however, `*` is more intuitive from a mathematics standpoint. `*` is usually used for non-commutative operations. For example, `*` is not commutative for matrices. Similarly, concatenating a string is not commutative. `Hello + World` should not be the same thing as `World + Hello`. Hence, the creators of Julia felt that `*` was a more appropriate operator.

In [22]:
str_1 = "Hel"
str_2 = "lo "
str_3 = "wor"
str_4 = "ld!"
println(str_1 * str_2 * str_3 * str_4)
println(str_1 * str_2 * str_3 * str_4 * "!!!")

Hello world!
Hello world!!!!


Julia also offers interpolation. Perl is known for being an incredibly easy language to work with strings with. The creators of Julia wanted similar syntax: `$`.

Rather than frequently using `str()` or `string()` like in python, they wanted a single character to denote that the expression should be converted to a string.

In [23]:
println("5 + 6 = $(5 + 6)")

5 + 6 = 11


That being said, Julia still offers the programmer the option to use `string()` if this syntax is preferable to you, although it is much longer to type.

In [24]:
println("5 + 6 = " * string(5 + 6))

5 + 6 = 11


Just to show equivalency:

In [25]:
s1 = str_1 * str_2 * str_3 * str_4
s2 = "$str_1$str_2$str_3$str_4"
s3 = string(str_1,str_2,str_3,str_4)

println(s1 === s2 === s3)

true


### Tuples, Arrays, and Dictionaries

Julia has mostly the same data types as Python (or other similar languages).

Tuples:

In [26]:
my_tuple = ("test", 1, 2.345, false)

println(my_tuple[1])
println(my_tuple[2])
println(my_tuple[3])
println(my_tuple[4])

test
1
2.345
false


(Tuples are immutable in Julia.)

Arrays:

Similarly to Python, arrays are mutable and can hold any type of element, including multiple element types in the same array.

They also have "push" and "pop" methods to add or take away from an array.

In [27]:
my_arr = ["test", 1, 2.345, false]

println(my_arr[1])
println(my_arr[2])
println(my_arr[3])
println(my_arr[4])

my_arr[3] = 6.789

push!(my_arr, "another string")
println(my_arr)

pop!(my_arr)
println(my_arr)

test
1
2.345
false
Any["test", 1, 6.789, false, "another string"]
Any["test", 1, 6.789, false]


Dictionaries:

In [28]:
dict = Dict("Key1" => "Value1", "Key2" => "Value2", "Key3" => "Value3")

println(dict)

dict["Key4"] = "Value4"

println(dict)



Dict("Key3" => "Value3", "Key2" => "Value2", "Key1" => "Value1")
Dict("Key3" => "Value3", "Key2" => "Value2", "Key1" => "Value1", "Key4" => "Value4")


Dictionaries also have the built in "pop" method.

In [29]:
out = pop!(dict, "Key1")
println(out)
println(dict)

Value1
Dict("Key3" => "Value3", "Key2" => "Value2", "Key4" => "Value4")


### Matrices

Working with matrices is very common in languages built around mathematics. One of the desired features of Julia was that it be as easy to do linear algebra with as MATLAB, which is short for "Matrix Laboratory." If you are familiar with MATLAB, much of this should look familiar to you.

As with most languages, "matrices" are actually multi-dimensional arrays, or in other words, arrays that are holding elements of the type "array".

In [30]:
array_array = [[1,2,3],[4,5,6],[7,8,9]]
println(array_array)

[[1, 2, 3], [4, 5, 6], [7, 8, 9]]


Julia  has many built-in functions for working with these, which we won't go over all of now since many are self explanatory, but here is a list of common ones also available with similar syntax in MATLAB:

* `length(matrix)`: the number of elements in the matrix
* `ndims(matrix)`: the number of dimensions in the matrix
* `size(matrix)`: a tuple containing the dimensions
* `size(matrix,n)`: the size along a specific dimension
* `axes(matrix)`: a tuple containing the valid indices of the matrix
* `axes(matrix,n)`: a range of valid indices along a specific dimension

In [31]:
println(ndims(array_array))

println(size(array_array,1))

1
3


Julia also has some fun built-in functions to help with matrix construction.

For example, declaring arrays with all zeros, arrays with random numbers, or diagonal matrices.

In [32]:
randArr = rand(2,3)
println(randArr)

zeroArr = zeros(2,3)
println(zeroArr)

[0.23671815383808714 0.5731603025549878 0.7948839623366752; 0.22090454841736784 0.8566880857960805 0.28086260342055636]
[0.0 0.0 0.0; 0.0 0.0 0.0]


LoadError: ignored

It also has a library, `Linear Algebra` to help with some more mathy stuff.

In [34]:
using LinearAlgebra

# Create an identity matrix
identityMatrix = LinearAlgebra.Matrix{Int8}(I, 5, 5)
println(identityMatrix)
println()
diagArr = Diagonal([1,2,3])
println(diagArr)

Int8[1 0 0 0 0; 0 1 0 0 0; 0 0 1 0 0; 0 0 0 1 0; 0 0 0 0 1]

[1 0 0; 0 2 0; 0 0 3]


Above is the syntax for how to include the library in your code and how to call specific functions within it.

This library includes hundreds of matrix operations, including creating upper triangular matrices, diagonal matrices, and all the matrix operations. It also includes several types of factorizations!

### Functions and Loops

Julia takes inspiration from MATLAB for it's function syntax. 

Functions are declared with the `function` keyword, followed by the name of the function and parenthesis. Unlike python, you don't need a colon to indicate the start of a function (or any other type of loop- just whitespace). You do, however, need to indicate where your loops and functions are ending. This is intuitively done with the keyword `end`.

Here are a few examples of functions, all of which print out the exact same thing. (You can comment them out and rerun if you want.)

In [35]:
function printNumsWhileLoop()
  n = 0
  while n < 10
    n += 1
    println(n)
  end
end

function printNumsForLoop()
  for i = 1:10
    println(i)
  end
end

function printNumsForElement()
  for i ∈ 1:10
    println(i)
  end
end

#printNumsWhileLoop()
printNumsForLoop()
#printNumsForElement()

1
2
3
4
5
6
7
8
9
10


In Julia, ranges are done using `start:stop`. The start value, followed by a colon, then the end value. Both the start and stop are inclusive (meaning if you start on 1 and end on 10, both 1 and 10 are included. Python excludes the stop value, so it would only print 1-9.) 

Here's an example of a nested for loop:

In [36]:
function additionTable(m,n)
  A = zeros(m,n)
  for i in 1:m 
    for j in 1:n 
      A[i, j] = i + j
    end
  end
  return A
end

additionTable(5,5)

5×5 Matrix{Float64}:
 2.0  3.0  4.0  5.0   6.0
 3.0  4.0  5.0  6.0   7.0
 4.0  5.0  6.0  7.0   8.0
 5.0  6.0  7.0  8.0   9.0
 6.0  7.0  8.0  9.0  10.0

Julia allows you to do both for loops in one line with a single `end` keyword.

In [37]:
function additionTable2(m,n)
  A = zeros(m,n)
  for i in 1:m, j in 1:n 
      A[i, j] = i + j
  end
  return A
end

additionTable2(5,5)

5×5 Matrix{Float64}:
 2.0  3.0  4.0  5.0   6.0
 3.0  4.0  5.0  6.0   7.0
 4.0  5.0  6.0  7.0   8.0
 5.0  6.0  7.0  8.0   9.0
 6.0  7.0  8.0  9.0  10.0

One more way using array comprehension (similarly to list comprehension in Python):

In [38]:
function additionTable3(m,n)
  return [i + j for i in 1:m, j in 1:n]
end

additionTable3(5,5)

5×5 Matrix{Int64}:
 2  3  4  5   6
 3  4  5  6   7
 4  5  6  7   8
 5  6  7  8   9
 6  7  8  9  10

### Conditionals

Julia also allows you to do conditionals very similarly to other programming languages. You use `if`, `elseif`, `else`, and `end` to indicate logical flows.

In [39]:
function conditional(m)
  if m === 0
    println("zero")
  elseif m > 0
    println("positive")
  else
    println("negative")
  end
end

conditional(0)
conditional(1)
conditional(-1)

zero
positive
negative


Julia also allows complex conditionals the same way many other languages do. `&&` indicates the logical "and", while `||` indicates the logical "or."

In [41]:
function complexConditional(m,n)
  if (m > 0) && (n > 0)
    println("both positive")
  elseif (m > 0) || (n > 0)
    println("one positive")
  else
    println("neither positive")
  end
end

complexConditional(2,3)
complexConditional(-2,3)
complexConditional(-2,-3)

both positive
one positive
neither positive


General note about functions: They are always pass-by-value in Julia as a default!!!

---

We hope you enjoyed this brief introduction to Julia! Please let us know in the feedback survey what else you would like to see or if we forgot anything!